In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

In [ ]:
# Загружаем датасет iris, определяем признаки и целевую переменную
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
iris.target_names

In [ ]:
# Оставляем 2 класса: 0 - 'versicolor', 1 - 'virginica'
X = iris.data[iris.target != 0]
y = iris.target[iris.target != 0] - 1
y

In [ ]:
X[:, 1]

Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. 

In [ ]:
# Для начала реализуем встроенный метод логистической регрессии

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучим модель встроенным модулем
model = LogisticRegression()
model.fit(X_train, y_train)

# И оценим качество модели
score = model.score(X_test, y_test)
print("Оценка точности модели:", score)

In [ ]:
# Получим коэффициенты
model.coef_, model.intercept_

In [ ]:
y_pred = model.predict(X_test)
data_results = pd.DataFrame({
    'Реальные значения': y_test,
    'Предсказанные значения': y_pred
})
print(data_results)

In [ ]:
# Теперь самостоятельно реализуем метод логистической регрессии методом градиентного спуска

# Масштабируем признаки 
scaler = StandardScaler()
X_scaled_features = scaler.fit_transform(X)

# Добавляем столбец единиц для свободного веса
X = np.c_[np.ones(len(X_scaled_features)), X_scaled_features]

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Функция сигмоиды
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Функция для вычисления логарифмической функции потерь
def logloss(y, y_proba):
    return -np.mean(y * np.log(y_proba + 1e-30) + (1 - y) * np.log(1 - y_proba + 1e-30))

# Градиентный спуск
def gr_logloss(X, W, y):
    y_proba = sigmoid(X @ W)
    grad = X.T @ (y_proba - y) / len(y)
    return grad

# Параметры обучения 
# learning_rate: скорость обучения, epochs: количество итераций, beta: скорость сглаживания
eps = 0.0001
learning_rate = 0.001
beta1 = 0.9
beta2 = 0.999
epochs = 10000

# Инициализация весов
W = np.zeros(X.shape[1])

for i in range(epochs):
    cur_W = W.copy()  # текущие веса
    
    # Градиент по текущим весам
    grad = gr_logloss(X, cur_W, y)
    
    # Обновляем веса
    W_next = cur_W - learning_rate * grad
    
    # Проверка условия остановки
    if np.linalg.norm(W - W_next) <= eps:
        print(f"Остановились на итерации {i}")
        break
    
    # Обновляем веса для следующей итерации
    W = W_next
    
    # Каждые 100 итераций выводим информацию
    if i % 100 == 0:
        y_proba = sigmoid(X @ W)
        y_class = (y_proba >= 0.5).astype(int)
        accuracy = (y_class == y).mean()
        print(f"Итерация {i}")
        print(f"Logloss: {logloss(y, y_proba):.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("--------------------------------------------------------")

In [ ]:
data_results = pd.DataFrame({
    'Реальные значения': y,
    'Предсказанные значения': y_class,
    'Предсказанные вероятности': y_proba
})
print(data_results)

In [ ]:
print("Финальные веса:", W)
i_grad = i
print("Итераций:", i_grad)
accuracy_grad = accuracy
print("Оценка точности модели методом градиентного спуска:", accuracy_grad)

Обучите логистическую регрессию этим методом скользящего среднего (Root Mean Square Propagation, RMSProp)

In [ ]:
# Реализуем метод логистической регрессии методом скользящего среднего

eps = 0.001
W = np.zeros(X.shape[1])
# Инициализация переменной для скользящего среднего квадрата градиентов
v = np.zeros_like(W)

for i in range(epochs):
    cur_W = W.copy()  # текущие веса
    
    # Градиент по текущим весам
    grad = gr_logloss(X, cur_W, y)
    
    # Обновляем v
    v = beta1 * v + (1 - beta1) * (grad ** 2)
    
    # Обновляем веса с учетом RMSProp
    W_next = cur_W - (learning_rate / (np.sqrt(v) + eps)) * grad
    
    # Проверка условия остановки
    if np.linalg.norm(W - W_next) <= eps:
        print(f"Остановились на итерации {i}")
        break       
    
    # Обновляем веса для следующей итерации
    W = W_next

    # Каждые 100 итераций выводим информацию
    if i % 100 == 0:
        y_proba = sigmoid(X @ W)
        y_class = (y_proba >= 0.5).astype(int)
        accuracy = (y_class == y).mean()
        print(f"Итерация {i}")
        print(f"Logloss: {logloss(y, y_proba):.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("--------------------------------------------------------")

In [ ]:
print("Финальные веса:", W)
i_RMSProp = i
print("Итераций:", i_RMSProp)
accuracy_RMSProp = accuracy
print("Оценка точности модели методом скользящего среднего:", accuracy_RMSProp)

Обучите логистическую регрессию этим методом адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam)

In [ ]:
# Реализуем метод логистической регрессии методом адаптивной оценки моментов

eps = 1e-8
W = np.zeros(X.shape[1])
# Инициализация переменных
m = np.zeros_like(W)
v = np.zeros_like(W)
t = 0

for i in range(epochs):
    t += 1

    grad = gr_logloss(X, cur_W, y)
    
    # Обновляем моменты
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)
    
    # Корректируем смещения для моментов
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    
    # Предсказанный момент для Nesterov
    m_bar = beta1 * m_hat + (1 - beta1) * grad / (1 - beta1 ** t)
    
    # Обновляем веса
    W_next = cur_W - learning_rate * m_bar / (np.sqrt(v_hat) + eps)
    
    # Проверка условия остановки
    if np.linalg.norm(W - W_next) <= eps:
        print(f"Остановились на итерации {i}")
        break       
    
    # Обновляем веса для следующей итерации
    W = W_next

    # Каждые 10 итераций выводим информацию
    if i % 10 == 0:
        y_proba = sigmoid(X @ W)
        y_class = (y_proba >= 0.5).astype(int)
        accuracy = (y_class == y).mean()
        print(f"Итерация {i}")
        print(f"Logloss: {logloss(y, y_proba):.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("--------------------------------------------------------")

In [ ]:
print("Финальные веса:", W)
i_Nadam = i
print("Итераций:", i_Nadam)
accuracy_Nadam = accuracy
print("Оценка точности модели методом адаптивной оценки моментов:", accuracy_Nadam)

Сравните значение метрик для реализованных методов оптимизации. Напишите вывод.

In [ ]:
data = {
    'Метод': ['Логистическая регрессия', 'Градиентный спуск', 'Метод скользящего среднего', 'Метод Нестерова'],
    'Метрика': [score, accuracy_grad, accuracy_RMSProp, accuracy_Nadam],
    'Время работы (итераций)': [1, i_grad, i_RMSProp, i_Nadam]
}

In [ ]:
data = pd.DataFrame(data)
print(data)

### Итоги:
На выходе мы имеем хорошие показатели метрики оценки качества модели по всем используемым методам. Метод Нестерова показывает самый быстрый расчет по сравнению с Градиентным спуском и Скользящим средним. Наилучшие показатели точности у Метода скользящего среднего и Нестерова. Подводя итог, можно сделать вывод, что скорость и полученная метрика зависят от настроек параметров. В частности, варьируя показателями скорости обучения и количеством итераций можно повысить метрику качества для Градиентного спуска до 0.97 (learning_rate = 0.01, epochs = 100000).

PS. Стоит также отметить, что в написанных функциях обучение происходит на всей выборке X (а в модуле LogisticRegression мы используем тренировочную выборку)